In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

### Wczytanie danych

In [61]:
matches = pd.read_csv("data/matches_with_rolling_stats_pl.csv")

### Metryka formacji - jedyny sensowny pomysł wiąże się ze sprawdzeniem czy drużyny grają z wahadłowymi czy bez

In [109]:
matches["formation_back_line"] = matches["formation_home"].apply(lambda text: int(text[0] == "4"))
# 1 - grają bez wahadłowych
# 0 - grają z wahadłowymi

In [41]:
# Count occurrences of outcomes for each formation match-up
outcome_counts = matches.groupby(
    ["formation_home", "formation_away", "outcome"]
).size().unstack(fill_value=0)

# Rename columns for clarity
outcome_counts.columns = ['draws', 'wins', 'losses']

# Reset index for easier manipulation
outcome_counts = outcome_counts.reset_index()
# Calculate win rate
outcome_counts['total_games'] = outcome_counts['wins'] + outcome_counts['losses'] + outcome_counts['draws']
outcome_counts['win_rate'] = outcome_counts['wins'] / outcome_counts['total_games']
outcome_counts['loss_rate'] = outcome_counts['losses'] / outcome_counts['total_games']
outcome_counts['draw_rate'] = outcome_counts['draws'] / outcome_counts['total_games']


In [51]:
outcome_counts[outcome_counts["total_games"] > 4].sort_values("loss_rate", ascending=False)

,formation_home,formation_away,draws,wins,losses,total_games,win_rate,loss_rate,draw_rate
192,5-3-2,4-2-3-1,0,0,7,7,0.000000,1.000000,0.000000
194,5-3-2,4-3-3,1,0,8,9,0.000000,0.888889,0.111111
184,4-5-1,4-3-3,0,2,9,11,0.181818,0.818182,0.000000
203,5-4-1,4-2-3-1,1,1,6,8,0.125000,0.750000,0.125000
17,3-4-1-2,4-2-3-1,1,1,4,6,0.166667,0.666667,0.166667
...,...,...,...,...,...,...,...,...,...
166,4-4-2,3-5-2,3,7,0,10,0.700000,0.000000,0.300000
150,4-3-3,5-4-1,2,11,0,13,0.846154,0.000000,0.153846
144,4-3-3,4-3-2-1,0,5,0,5,1.000000,0.000000,0.000000
139,4-3-3,4-1-2-1-2,0,5,0,5,1.000000,0.000000,0.000000


### Analiza danych - korelacje

In [11]:
matches.describe()

,round,attendance_value,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,...,away_last30_fouled,away_last30_offsides,away_last30_pens_won,away_last30_pens_conceded,away_last30_own_goals,away_last30_ball_recoveries,away_last30_aerials_won,away_last30_aerials_lost,away_last30_aerials_won_pct,away_last30_points
count,2100.00000,1689.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.00000,2100.000000,2100.000000,...,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000
mean,21.00000,38529.040853,988.268571,1.562381,1.053810,0.131429,0.152857,13.76619,4.549048,1.675238,...,10.286980,1.811502,0.115381,0.143164,0.053718,52.390467,16.100583,16.083826,50.031870,1.387290
std,10.10191,17002.670205,9.652330,1.334275,1.097643,0.359798,0.390412,5.73814,2.532302,1.291765,...,1.628433,0.490567,0.112875,0.115526,0.060351,3.574435,3.837091,3.892820,3.856789,0.573937
min,4.00000,2000.000000,888.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,5.333333,0.000000,0.000000,0.000000,0.000000,40.333333,6.666667,5.666667,35.133333,0.000000
25%,12.00000,25541.000000,990.000000,1.000000,0.000000,0.000000,0.000000,10.00000,3.000000,1.000000,...,9.250000,1.500000,0.033333,0.066667,0.000000,49.914773,13.326923,13.400000,47.593333,1.000000
50%,21.00000,32485.000000,990.000000,1.000000,1.000000,0.000000,0.000000,13.00000,4.000000,2.000000,...,10.250000,1.800000,0.100000,0.133333,0.040000,52.631481,15.500000,15.783333,49.965000,1.333333
75%,30.00000,53177.000000,990.000000,2.000000,2.000000,0.000000,0.000000,17.00000,6.000000,2.000000,...,11.333333,2.133333,0.166667,0.200000,0.083333,54.800000,18.346154,18.565580,52.295833,1.750000
max,38.00000,81332.000000,990.000000,9.000000,6.000000,2.000000,2.000000,36.00000,15.000000,7.000000,...,16.333333,4.333333,1.000000,1.500000,0.500000,64.800000,31.714286,28.600000,66.660000,3.000000


In [20]:
correlation_matrix

,round,home_minutes,home_goals,home_assists,home_pens_made,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,...,away_last30_dispossessed,away_last30_cards_yellow_red,away_last30_fouls,away_last30_fouled,away_last30_pens_conceded,away_last30_own_goals,away_last30_ball_recoveries,away_last30_aerials_won,away_last30_aerials_lost,away_last30_aerials_won_pct
round,1.000000,0.042948,0.016542,0.011644,0.004457,0.026005,0.027768,0.079298,0.035434,0.017346,...,0.073717,0.031764,0.041523,0.013117,0.081546,0.053684,0.144046,0.063732,0.070358,0.010550
home_minutes,0.042948,1.000000,0.077492,0.074944,0.025533,0.142598,0.107345,0.077444,0.793479,0.145989,...,0.003159,0.009131,0.011592,0.034862,0.002842,0.024540,0.021630,0.013151,0.009246,0.008226
home_goals,0.016542,0.077492,1.000000,0.842668,0.269715,0.326321,0.586933,0.127829,0.075525,0.215000,...,0.008363,0.055140,0.095035,0.045726,0.032899,0.051786,0.015717,0.042265,0.095037,0.095867
home_assists,0.011644,0.074944,0.842668,1.000000,0.007417,0.284970,0.515050,0.122742,0.080856,0.203817,...,0.004178,0.047453,0.063423,0.022811,0.013900,0.053717,0.014888,0.055408,0.096347,0.077119
home_pens_made,0.004457,0.025533,0.269715,0.007417,1.000000,0.033352,0.041551,0.001675,0.028041,0.052503,...,0.032062,0.025801,0.040293,0.022927,0.003390,0.048793,0.016364,0.015137,0.016096,0.056161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
away_last30_own_goals,0.053684,0.024540,0.051786,0.053717,0.048793,0.064478,0.036903,0.007667,0.021853,0.061447,...,0.027462,0.015030,0.084912,0.100025,0.028700,1.000000,0.098885,0.070028,0.006841,0.106338
away_last30_ball_recoveries,0.144046,0.021630,0.015717,0.014888,0.016364,0.087723,0.054817,0.056782,0.029861,0.090250,...,0.253518,0.053074,0.056676,0.219093,0.147282,0.098885,1.000000,0.113837,0.070911,0.060310
away_last30_aerials_won,0.063732,0.013151,0.042265,0.055408,0.015137,0.107746,0.049731,0.140583,0.020041,0.258234,...,0.047123,0.041200,0.130899,0.140386,0.062386,0.070028,0.113837,1.000000,0.834647,0.210424
away_last30_aerials_lost,0.070358,0.009246,0.095037,0.096347,0.016096,0.147913,0.111128,0.146697,0.021013,0.280585,...,0.050882,0.023409,0.138205,0.079919,0.021851,0.006841,0.070911,0.834647,1.000000,0.332556


### Regresja logistyczna do feature selection

In [25]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [30]:
matches = pd.read_csv("matches_with_rolling_stats_pl.csv")
matches = matches[matches['round'] > 7]
matches.head(3)
rolling_stats = matches[[col for col in matches.columns if 'last' in col or 'outcome' in col]]

In [41]:
X = rolling_stats.drop(columns=["outcome"])  # Features
y = rolling_stats["outcome"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
log_reg = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr')
log_reg.fit(X_train, y_train)
model = SelectFromModel(log_reg, prefit=True)
selected_features = X.columns[model.get_support()]
print("Selected Features based on Logistic Regression:")
print(selected_features)
X_selected = model.transform(X_scaled)

Selected Features based on Logistic Regression:
Index(['home_last5_wins', 'home_last5_draws', 'home_last5_defeats',
       'home_last5_goals_conceded', 'home_last5_minutes', 'home_last5_goals',
       'home_last5_assists', 'home_last5_pens_made', 'home_last5_pens_att',
       'home_last5_shots',
       ...
       'away_last30_fouls', 'away_last30_fouled', 'away_last30_offsides',
       'away_last30_pens_won', 'away_last30_pens_conceded',
       'away_last30_own_goals', 'away_last30_ball_recoveries',
       'away_last30_aerials_won', 'away_last30_aerials_lost',
       'away_last30_aerials_won_pct'],
      dtype='object', length=339)


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Assuming `X_selected` contains only the selected features, and `y` is the target variable
# Re-split the data if necessary to align with selected features
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Initialize and train the logistic regression model on selected features
log_reg_model = LogisticRegression(multi_class='ovr', solver='liblinear')
log_reg_model.fit(X_selected, y)

# Make predictions on the test set
y_pred = log_reg_model.predict(X_selected)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y, y_pred, target_names=["Draw", "Win", "Lose"]))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y, y_pred))

Accuracy: 0.66

Classification Report:
              precision    recall  f1-score   support

        Draw       0.59      0.36      0.45       404
         Win       0.68      0.79      0.73       841
        Lose       0.66      0.67      0.66       615

    accuracy                           0.66      1860
   macro avg       0.64      0.61      0.61      1860
weighted avg       0.65      0.66      0.65      1860


Confusion Matrix:
[[147 169  88]
 [ 49 667 125]
 [ 52 152 411]]


In [44]:
# Get predicted probabilities for the test set
y_pred_proba = log_reg_model.predict_proba(X_selected)

# Example of printing probabilities for the first few predictions
for i, probs in enumerate(y_pred_proba[:5]):
    print(f"Sample {i + 1} - Draw: {probs[0]:.2f}, Win: {probs[1]:.2f}, Lose: {probs[2]:.2f}")


Sample 1 - Draw: 0.24, Win: 0.28, Lose: 0.48
Sample 2 - Draw: 0.07, Win: 0.66, Lose: 0.27
Sample 3 - Draw: 0.03, Win: 0.76, Lose: 0.22
Sample 4 - Draw: 0.03, Win: 0.16, Lose: 0.81
Sample 5 - Draw: 0.12, Win: 0.83, Lose: 0.06


In [46]:
matches.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,total_goals,Stadium,Club,Location,Capacity,Pitch length m,Pitch width m,latitude,longitude,outcome,home_points,away_points,home_last5_m

## Metryka zmęczenia zależna od ilości dni od ostatniego meczu + Metryka nowego menadżera
Trzeba dodać rozgrywki europejskie
Można ustawić od kiedy są menadżerowie na początku pierwszego sezonu danych, ale nie powinno to nic specjalnie zmienić

In [62]:
def calculate_tiredness_factor(days_since_last_match, decay_rate=0.1):
    # Higher decay rate means quicker recovery. Adjust as needed.
    return np.exp(-decay_rate * days_since_last_match)

In [63]:
matches['last_match_date'] = pd.NaT
matches['date'] = pd.to_datetime(matches['date'])
matches = matches.sort_values(by='date', ignore_index=True)
# Iterate through each match row
for i, row in matches.iterrows():
    # Find the home team and away team
    home_team = row['home_team']
    away_team = row['away_team']
    
    # Initialize last match date to NaT (Not a Time) for both teams
    last_home_date = pd.NaT
    last_away_date = pd.NaT

    home_team_last_manager = None
    away_team_last_manager = None
    
    # For home team: Search for the last match where the team played (either home or away)
    for j in range(i-1, -1, -1):  # Iterate backward from the current match
        if matches.iloc[j]['home_team'] == home_team:
            last_home_date = matches.iloc[j]['date']
            home_team_last_manager = matches.iloc[j]["home_manager"]
            break  # Stop once the match is found
        if matches.iloc[j]['away_team'] == home_team:
            last_home_date = matches.iloc[j]['date']
            home_team_last_manager = matches.iloc[j]["away_manager"]
            break
    
    # For away team: Search for the last match where the team played (either home or away)
    for j in range(i-1, -1, -1):  # Iterate backward from the current match
        if matches.iloc[j]['home_team'] == away_team:
            last_away_date = matches.iloc[j]['date']
            away_team_last_manager = matches.iloc[j]["home_manager"]
            break  # Stop once the match is found
        if matches.iloc[j]['away_team'] == away_team:
            last_away_date = matches.iloc[j]['date']
            away_team_last_manager = matches.iloc[j]["away_manager"]
            break

    home_matches_since_last_manager = None
    away_matches_since_last_manager = None
    if home_team_last_manager == None:
        home_team_last_manager = row["home_manager"]
        home_matches_since_last_manager = 20
    if away_team_last_manager == None:
        away_team_last_manager = row["away_manager"]
        away_matches_since_last_manager = 20

    if home_team_last_manager == row["home_manager"]:
        new_home_manager = False
    else:
        new_home_manager = True

    if away_team_last_manager and away_team_last_manager == row["away_manager"]:
        new_away_manager = False
    else:
        new_away_manager = True

    # Assign the last match date for home and away teams
    matches.at[i, 'last_match_date_home'] = last_home_date
    matches.at[i, 'last_match_date_away'] = last_away_date
    matches.at[i, 'last_home_manager'] = home_team_last_manager
    matches.at[i, 'last_away_manager'] = away_team_last_manager
    matches.at[i, 'new_home_manager'] = new_home_manager
    matches.at[i, 'new_away_manager'] = new_away_manager
    matches.at[i, 'matches_since_new_home_manager'] = home_matches_since_last_manager
    matches.at[i, 'matches_since_new_away_manager'] = away_matches_since_last_manager    

matches['days_since_last_home'] = (matches['date'] - matches['last_match_date_home']).dt.days
matches['days_since_last_away'] = (matches['date'] - matches['last_match_date_away']).dt.days
matches['days_since_last_home'] = matches['days_since_last_home'].fillna(7)
matches['days_since_last_away'] = matches['days_since_last_away'].fillna(7)
matches['home_tiredness'] = calculate_tiredness_factor(matches['days_since_last_home'])
matches['away_tiredness'] = calculate_tiredness_factor(matches['days_since_last_away'])


matches[(matches['home_team'] == "Burnley") | (matches['away_team'] == "Burnley")][["home_team", "away_team", "date", "last_match_date_home", "last_match_date_away", "days_since_last_away", "days_since_last_home", "home_tiredness", "away_tiredness"]].head(6)

,home_team,away_team,date,last_match_date_home,last_match_date_away,days_since_last_away,days_since_last_home,home_tiredness,away_tiredness
9,Burnley,Manchester United,2018-09-02,NaT,NaT,7.0,7.0,0.496585,0.496585
17,Wolverhampton Wanderers,Burnley,2018-09-16,2018-09-01,2018-09-02,14.0,15.0,0.223130,0.246597
25,Burnley,Bournemouth,2018-09-22,2018-09-16,2018-09-15,7.0,6.0,0.548812,0.496585
38,Cardiff City,Burnley,2018-09-30,2018-09-22,2018-09-22,8.0,8.0,0.449329,0.449329
46,Burnley,Huddersfield Town,2018-10-06,2018-09-30,2018-09-29,7.0,6.0,0.548812,0.496585
53,Manchester City,Burnley,2018-10-20,2018-10-07,2018-10-06,14.0,13.0,0.272532,0.246597


In [ ]:
matches

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,last_home_manager,last_away_manager,new_home_manager,new_away_manager,matches_since_new_home_manager,matches_since_new_away_manager,days_since_last_home,days_since_last_away,home_tiredness,away_tiredness
0,2018-2019,2018-09-01,12:30,4,32149.0,Paul Tierney,Claude Puel,Jürgen Klopp,Wes Morgan,Jordan Henderson,...,Claude Puel,Jürgen Klopp,False,False,20.0,20.0,7.0,7.0,0.496585,0.496585
1,2018-2019,2018-09-01,15:00,4,25495.0,Martin Atkinson,Roy Hodgson,Mark Hughes,Luka Milivojević,Ryan Bertrand,...,Roy Hodgson,Mark Hughes,False,False,20.0,20.0,7.0,7.0,0.496585,0.496585


In [69]:
for i in range (len(matches)):
    # Find the home team and away team
    home_team = matches.iloc[i]['home_team']
    away_team = matches.iloc[i]['away_team']

    new_home_manager = matches.iloc[i]["new_home_manager"]
    new_away_manager = matches.iloc[i]["new_away_manager"]

    if new_home_manager:
        matches.at[i, 'matches_since_new_home_manager'] = 0
    if new_away_manager:
        matches.at[i, 'matches_since_new_away_manager'] = 0

    for j in range(i+1, len(matches)):
        if matches.iloc[j]['home_team'] == home_team:
            matches.at[j, 'matches_since_new_home_manager'] = matches.iloc[i]["matches_since_new_home_manager"] + 1
            break
        if matches.iloc[j]['away_team'] == home_team:
            matches.at[j, 'matches_since_new_away_manager'] = matches.iloc[i]["matches_since_new_home_manager"] + 1
            break

    for j in range(i+1, len(matches)):
        if matches.iloc[j]['home_team'] == away_team:
            matches.at[j, 'matches_since_new_home_manager'] = matches.iloc[i]["matches_since_new_away_manager"] + 1
            break
        if matches.iloc[j]['away_team'] == away_team:
            matches.at[j, 'matches_since_new_away_manager'] = matches.iloc[i]["matches_since_new_away_manager"] + 1
            break
    
    

In [78]:
matches[230:].head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,last_home_manager,last_away_manager,new_home_manager,new_away_manager,matches_since_new_home_manager,matches_since_new_away_manager,days_since_last_home,days_since_last_away,home_tiredness,away_tiredness
230,2018-2019,2019-02-11,20:00,26,30687.0,Graham Scott,Nuno Espírito Santo,Rafael Benítez,Conor Coady,Jamaal Lascelles,...,Nuno Espírito Santo,Rafael Benítez,False,False,42.0,42.0,9.0,9.0,0.406570,0.406570
231,2018-2019,2019-02-22,19:45,27,59950.0,Lee Mason,Manuel Pellegrini,Claudio Ranieri,Mark Noble,Tom Cairney,...,Manuel Pellegrini,Claudio Ranieri,False,False,43.0,14.0,13.0,13.0,0.272532,0.272532


## Metryki H2H

In [3]:
matches[(matches['away_team'] == "Everton") & (matches["home_team"] == "Sheffield United")][["home_team", "outcome"]]

,home_team,outcome
1,Sheffield United,0
1163,Sheffield United,2
1734,Sheffield United,2


In [4]:
def get_h2h_metrics(row, df, num_matches=6):
    past_h2h = df[((df['home_team'] == row['home_team']) & (df['away_team'] == row['away_team'])) |
                  ((df['home_team'] == row['away_team']) & (df['away_team'] == row['home_team']))]
    past_h2h = past_h2h[past_h2h['date'] < row['date']].sort_values(by='date', ascending=False)
    
    past_h2h = past_h2h.head(num_matches)
    num_past_h2h = len(past_h2h)

    past_h2h_home_home = past_h2h[past_h2h["home_team"] == row["home_team"]]
    past_h2h_home_away = past_h2h[past_h2h["home_team"] == row["away_team"]]

    if not past_h2h.empty:
        metrics = {
            'h2h_win_ratio': ((past_h2h_home_home['outcome'] == 1).sum() + (past_h2h_home_away['outcome'] == 2).sum()) / num_past_h2h,
            'h2h_draw_ratio': (past_h2h['outcome'] == 0).sum() / num_past_h2h,
            'h2h_avg_goals_scored_home_team': (past_h2h_home_home['home_goals'].sum() + past_h2h_home_away['away_goals'].sum()) / num_past_h2h,
            'h2h_avg_goals_scored_away_team': (past_h2h_home_home['away_goals'].sum() + past_h2h_home_away['home_goals'].sum()) / num_past_h2h,
            'h2h_avg_xG_home_team': (past_h2h_home_home['home_xg'].sum() + past_h2h_home_away['away_xg'].sum()) / num_past_h2h,
            'h2h_avg_xG_away_team': (past_h2h_home_home['away_xg'].sum() + past_h2h_home_away['home_xg'].sum()) / num_past_h2h,
            # Add other metrics as needed
        }
    else:
        metrics = {
            'h2h_win_ratio': None,
            'h2h_draw_ratio': None,
            'h2h_avg_goals_scored_home_team': None,
            'h2h_avg_goals_scored_away_team': None,
            'h2h_avg_xG_home_team': None,
            'h2h_avg_xG_away_team': None,
        }

    return pd.Series(metrics)

matches_with_h2h = matches.join(matches.apply(lambda row: get_h2h_metrics(row, matches), axis=1))

In [5]:
matches_with_h2h.head(4)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,...,away_last5_aerials_won,away_last5_aerials_lost,away_last5_aerials_won_pct,away_last5_points,h2h_win_ratio,h2h_draw_ratio,h2h_avg_goals_scored_home_team,h2h_avg_goals_scored_away_team,h2h_avg_xG_home_team,h2h_avg_xG_away_team
0,2023-2024,2023-09-01,20:00,4,10802.0,Paul Tierney,Rob Edwards,David Moyes,Carlton Morris,Kurt Zouma,...,17.666667,16.000000,50.366667,2.333333,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-2024,2023-09-02,12:30,4,31124.0,Andy Madley,Paul Heckingbottom,Sean Dyche,John Egan,James Tarkowski,...,10.333333,10.666667,49.033333,0.000000,0.5,0.0,0.750000,0.500000,0.90,0.925
2,2023-2024,2023-09-02,15:00,4,39820.0,Tim Robinson,Mauricio Pochettino,Steve Cooper,Ben Chilwell,Joe Worrall,...,23.666667,14.333333,62.700000,1.000000,0.0,1.0,1.500000,1.500000,1.05,1.350
3,2023-2024,2023-09-02,15:00,4,52899.0,Michael Oliver,Pep Guardiola,Marco Silva,Kyle Walker,Tom Cairney,...,13.333333,10.000000,57.266667,1.333333,1.0,0.0,2.333333,0.333333,2.75,0.450


## Zawodnicy

In [126]:
players = pd.read_csv("data/players_pl_17-18_fbref.csv")
fifa = pd.read_csv("data/sofifa_players_17_18.csv")

In [127]:
players.head(2)

,team,where,date,round,season,player,shirtnumber,nationality,position,age,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Arsenal,home,2017-08-11,1,2017-2018,Alexandre Lacazette,9,FRA,"FW,LW,LM",26-075,...,2,2,2,0,0,0,2,1,2,33.3
1,Arsenal,home,2017-08-11,1,2017-2018,Danny Welbeck,23,ENG,"AM,FW",26-258,...,1,1,1,0,0,0,2,1,1,50.0


In [145]:
def longest_two_substrings(s1, s2):
    # Initialize the matrix for storing common substring lengths
    matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]
    longest_substrings = []
    
    # Populate the matrix
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                matrix[i][j] = matrix[i - 1][j - 1] + 1
                if matrix[i][j] > 1:  # Only consider substrings longer than 1
                    longest_substrings.append(s1[i - matrix[i][j]:i])
            else:
                matrix[i][j] = 0

    # Sort substrings by length and pick the top 2 longest ones
    unique_substrings = sorted(set(longest_substrings), key=len, reverse=True)
    return unique_substrings[:2]
def similarity_score(name1, name2):
    substrings = longest_two_substrings(name1, name2)
    return sum(len(sub) for sub in substrings)

In [146]:
def find_best_match_with_longest_substrings(name, choices):
    scores = {choice: similarity_score(name, choice) for choice in choices}
    best_match = max(scores, key=scores.get)  # Find the match with the highest score
    return best_match

In [161]:
fifa_names = fifa["name"].unique()

# find_best_match_with_longest_substrings(players["player"][1], fifa_names)
players["player_name"] = players["player"].apply(lambda name: find_best_match_with_longest_substrings(name, fifa_names))

players[["player_name", "player"]]

,player_name,player
0,Alexandre Lacazette,Alexandre Lacazette
1,Daniel Nii Tackie Mensah Welbeck,Danny Welbeck
2,Theo Walcott,Theo Walcott
3,Mesut Özil,Mesut Özil
4,Granit Xhaka,Granit Xhaka
...,...,...
10443,Jacob Harry Maguire,Harry Maguire
10444,Wes Morgan,Wes Morgan
10445,Danny Simpson,Danny Simpson
10446,Hamza Choudhury,Hamza Choudhury


In [169]:
find_best_match_with_longest_substrings(players["player"][10349], fifa_names)

'Martin Braithwaite Christensen'